<a href="https://colab.research.google.com/github/RpicciniCorebi/Coe/blob/master/Prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install looker_sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.1/348.1 kB 6.2 MB/s eta 0:00:00


In [2]:
import looker_sdk #Note that the pip install required a hyphen but the import is an underscore.

import os #We import os here in order to manage environment variables for the tutorial. You don't need to do this on a local system or anywhere you can more conveniently set environment variables.

import json #This is a handy library for doing JSON work.

os.environ["LOOKERSDK_BASE_URL"] = "https://corebipartner.cloud.looker.com" #If your looker URL has .cloud in it (hosted on GCP), do not include :19999 (ie: https://your.cloud.looker.com).
os.environ["LOOKERSDK_API_VERSION"] = "4.0" #3.1 is the default version. You can change this to 4.0 if you want.
os.environ["LOOKERSDK_VERIFY_SSL"] = "true" #Defaults to true if not set. SSL verification should generally be on unless you have a real good reason not to use it. Valid options: true, y, t, yes, 1.
os.environ["LOOKERSDK_TIMEOUT"] = "120" #Seconds till request timeout. Standard default is 120.

#Get the following values from your Users page in the Admin panel of your Looker instance > Users > Your user > Edit API keys. If you know your user id, you can visit https://your.looker.com/admin/users/<your_user_id>/edit.
os.environ["LOOKERSDK_CLIENT_ID"] =  "Zg5ZXV9jgRkS4PcF5KJ6" #No defaults.
os.environ["LOOKERSDK_CLIENT_SECRET"] = "zPBPXwwbpBF5K4HjMdq3R2yg" #No defaults. This should be protected at all costs. Please do not leave it sitting here, even if you don't share this document.

print("All environment variables set.")


All environment variables set.


In [3]:
sdk = looker_sdk.init40()  # or init31() for the older v3.1 API
my_user = sdk.me()

# output can be treated like a dictionary
print(my_user["first_name"])

# or a model instance (User in this case)
print(my_user.last_name)

Rodrigo
Piccini


In [5]:
#Enter your Look ID. If your URL is https://your.cloud.looker.com/looks/25, your Look ID is 25.
look_id = '46'
look = sdk.look(look_id=look_id) 
# This gives us a Look object. We'll print the ID of it to verify everything's working.

print(look.id)

#You actually don't need to do anything further for this case, using a Look. 
#If you wanted to use an Explore instead, you'd have to get the underlying query first, which might look like this:

#explore_id = "Q4pXny1FEtuxMuj9Atf0Gg" 
#If your URL looks like https://your.cloud.looker.com/explore/ecommerce_data/order_items?qid=Q4pXny1FEtuxMuj9Atf0Gg&origin_space=15&toggle=vis, your explore_id/QID is Q4pXny1FEtuxMuj9Atf0Gg.
#explore_query = sdk.query_for_slug(slug=explore_id)

#This would return a Query object that we could then run to get results in step 2 using the run_query endpoints.

46


In [6]:
#We'll use a try/except block here, to make debugging easier. 
#In general, this kind of thing isn't really necessary in notebooks as each cell is already isolated from the rest,
#but it's a good practice in larger scripts and certainly in applications where fatal errors can break the entire app.
#You should get into the habit of using them.

try:
  response = sdk.run_look(
    look_id=look.id,
    result_format= "json" # Options here are csv, json, json_detail, txt, html, md, xlsx, sql (returns the raw query), png, jpg. JSON is the easiest to work with in python, so we return it.
  )
  data = json.loads(response) #The response is just a string, so we have to use the json library to load it as a json dict.
  print(data) #If our query was successful we should see an array of rows.
except:
  raise Exception(f'Error running look {look.id}')


[{'products.brand': ''}, {'products.brand': '!it Jeans'}, {'products.brand': "'47 Brand"}, {'products.brand': '007Lingerie'}, {'products.brand': '106Shades'}, {'products.brand': '12XLShop Inc.'}, {'products.brand': '180s'}, {'products.brand': '1826'}, {'products.brand': '1WorldSarong'}, {'products.brand': '1veMoon'}, {'products.brand': '1vemoon'}, {'products.brand': '2(x)ist'}, {'products.brand': '2117 of Sweden'}, {'products.brand': '2XU'}, {'products.brand': '2b by bebe'}, {'products.brand': '35 Degree Below'}, {'products.brand': '4-rth'}, {'products.brand': '5 Seasons'}, {'products.brand': '525 America'}, {'products.brand': '555 Turnpike'}, {'products.brand': '55DSL'}, {'products.brand': '626 BLUE'}, {'products.brand': '66 North'}, {'products.brand': '7 Diamonds'}, {'products.brand': '7 For All Mankind'}, {'products.brand': '89th &#x0026; Madison'}, {'products.brand': 'A Christmas Story'}, {'products.brand': 'A Pea in the Pod'}, {'products.brand': 'A shore Fit'}, {'products.brand': 

In [7]:
#Before we move on, here's a simple example of that. Let's print the first 10 rows.
#This script is set up to always only look at the first column, assuming our Look returns 1 column.
first_field = list(
    data[0].keys()
    )[0] #This looks at the first row of the data and returns the first field name. keys() returns a set, so we wrap it in list() to return an array.
    
for i in range(0,10):
  print(i,data[i][first_field])

#If we _know_ the name of the first field, why did we go to all this list(data[0].keys()[0]) trouble? Well, we know the name of the first field for ONE look. 
#This little trickery above makes it so that our script will always work for any Look, no matter what the name is, without having to edit the code.

0 
1 !it Jeans
2 '47 Brand
3 007Lingerie
4 106Shades
5 12XLShop Inc.
6 180s
7 1826
8 1WorldSarong
9 1veMoon
